In [ ]:
!pwd
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
'''
sys.path.append('/Users/suryaven/Documents/personal/sky/skyburst')
sys.path.append('/Users/suryaven/Documents/personal/sky/starburst/')
sys.path.append('/Users/suryaven/Documents/personal/sky/starburst/starburst/utils')
'''
# TODO: Generalize differences between sky laptop and personal latop paths
sys.path.append('/Users/surya/Documents/sky/skyburst/')
sys.path.append('/Users/surya/Documents/sky/starburst/')
sys.path.append('/Users/surya/Documents/sky/starburst/utils')

import starburst
import skyburst
from starburst import utils
import job_logs
import sampled_jobs
import plot_jobs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import time 
import copy 
from collections import OrderedDict 
pd.set_option('display.max_columns', None)

# TODO: Store data for all jobs into one single pandas dataframe
# TODO: Store the results of parse_event_logs with associated hyperparameters into single df 
# TODO: Attach batch hyperparameters associated with each job based on batch it ran on

In [ ]:
LOGS = {
    "05_01_2023": 1682925843,
    "05_02_2023": 1683099273,
    "05_03_2023": 1683132152,
    "05_03_2023_2": 1683173965,
    "05_04_2023_1": 1683184792, 
    "05_04_2023_2": 1683196845,
    "05_04_2023_3": 1683259533,
    "05_05_2023_4": 1683343491,
    "05_06_2023_4": 1683410859,
    "05_06_2023_5": 1683418719,
    "05_06_2023_6": 1683442154,
    "05_06_2023_7": 1683452077,
    "05_07_2023_1": 1683486242,
    "05_07_2023_2": 1683489564,
    "05_07_2023_3": 1683496107,
    "05_07_2023_4": 1683497538,
    "1683498468": 1683497538, 
    "1683498857": 1683498857,
    "1683499283": 1683499283,
    "1683528723": 1683528723, # Fixed JCT values
    "1683534589": 1683534589, # Large sweep with greater arrival rates
}

logs = LOGS["1683534589"]

job_logs.pull_vm_scheduler_logs(logs)
jobs_df = job_logs.retrieve_df(event_number=logs, graph=False, avoid_congestion=False)
jobs_df = jobs_df.assign(gpus_per_node=jobs_df['cpus_per_node'])

In [ ]:
def compute_total_time(row):
    total_time = [row['wait_times'][i] + row['runtime'][i] for i in range(len(row['wait_times']))]
    return total_time

jobs_df['total_time'] = jobs_df.apply(compute_total_time, axis=1)

def compute_completion_time(row):
    completion_time = [row['total_time'][i] + row['submission_time'][i] for i in range(len(row['wait_times']))]
    return completion_time

jobs_df['completion_time'] = jobs_df.apply(compute_completion_time, axis=1)
jobs_df['avg_jct'] = jobs_df['total_time'].apply(lambda x: sum(x)/len(x))

def compute_cluster_utilization(row):
    surface_area = [row['runtime'][i] * row['cpus'][i] for i in range(len(row['runtime']))]
    utilized_surface_area = sum(surface_area)
    total_surface_area = (max(row['completion_time']) - min(row['submission_time'])) * (row['cpus_per_node'] * row['cluster_size'])
    cluster_utilization = utilized_surface_area/total_surface_area
    return cluster_utilization

jobs_df['cluster_utilization'] = jobs_df.apply(compute_cluster_utilization, axis=1)

def compute_system_utilization(row):
    # TODO: Compute this value correctly
    return system_utilization

#jobs_df['cluster_utilization'] = jobs_df.apply(compute_cluster_utilization, axis=1)

In [ ]:
def sum_arrays(row):
    new_row = [row['wait_times'][i] + row['runtime'][i] for i in range(len(row['wait_times']))]
    return new_row

jobs_df['total_time'] = jobs_df.apply(sum_arrays, axis=1)
jobs_df['avg_wait'] = jobs_df['wait_times'].apply(lambda x: sum(x)/len(x))
jobs_df['avg_runtime'] = jobs_df['runtime'].apply(lambda x: sum(x)/len(x))
jobs_df['avg_jct'] = jobs_df['total_time'].apply(lambda x: sum(x)/len(x))
#jobs_df

jobs_wait = jobs_df.filter(['waiting_policy', 'wait_time', 'wait_times', 'arrival', 'start', 'submission_time', 'arrival_rate'])
jobs_wait['arrival_mask'] = jobs_df['start'].apply(lambda x: [0 if not i else 1 for i in x])

k8s_scheduling_waiting_constant = 1


#print("JOB WAIT BEFORE")
#print(jobs_wait['wait_times'])

onprem_waits = {}
cloud_waits = {}
new_waits = {}
for j in range(len(jobs_wait)):
    onprem_wait = [jobs_wait['wait_times'][j][i] for i, n in enumerate(jobs_wait['arrival_mask'][j]) if n == 1]
    #cloud_wait = [jobs_wait['wait_times'][j][i] for i, n in enumerate(jobs_wait['arrival_mask'][j]) if n == 0]
    #print("CLOUD WAIT BEFORE")
    #print(cloud_wait)
    #cloud_wait = [jobs_wait['wait_times'][j][i] + k8s_scheduling_waiting_constant + jobs_wait['wait_time'][j] for i, n in enumerate(jobs_wait['arrival_mask'][j]) if n == 0]
    cloud_wait = [k8s_scheduling_waiting_constant + jobs_wait['wait_time'][j] for i, n in enumerate(jobs_wait['arrival_mask'][j]) if n == 0]
    #print("CLOUD WAIT AFTER")
    #print(cloud_wait)
    
    #cloud_waits[j] = cloud_wait + jobs_wait['wait_time'] + k8s_scheduling_waiting_constant
    
    new_wait = onprem_wait + cloud_wait
    new_waits[j] = new_wait
    onprem_waits[j] = onprem_wait
    cloud_waits[j] = cloud_wait
    
    #new_wait[0] = 0
    
    jobs_wait['wait_times'][j] = new_wait
    #jobs_df['wait_times'][j] = new_wait
    #jobs_df.loc[j, 'wait_times'] = new_wait
jobs_df['wait_times'] = new_waits

    
    
#print("JOB WAIT AFTER")
#print(jobs_wait['wait_times'])

#print(onprem_wait)
#print(cloud_wait)
'''
print("NEW WAITS")
print(new_waits)
print("CLOUD WAITS")
print(cloud_waits)
print("ONPREM WAITS")
print(onprem_waits)
'''
#time.sleep(100000)
#jobs_wait['waitimes']

fig, axs = plt.subplots(nrows=1, ncols=len(jobs_wait), figsize=(150, 3)) 
for i in range(len(onprem_waits)):
    #TODO: Incorporate tracing for efficiently
    #import pdb; pdb.set_trace()
    #TODO: Plot separater graphs for onprem and cloud
    try: 
        axs[i].set_title("ONPREM " + "wait " + str(jobs_wait['wait_time'][i]) +  " arr " + str(jobs_wait['arrival_rate'][i]), fontsize=10)
        axs[i].hist(onprem_waits[i])
        print("Mask")
        print(jobs_wait['arrival_mask'][i])
    except Exception as e: 
        continue 

fig, axs = plt.subplots(nrows=1, ncols=len(jobs_wait), figsize=(50, 3)) 
for i in range(len(cloud_waits)):
    #TODO: Incorporate tracing for efficiently
    #import pdb; pdb.set_trace()
    # TODO: Plot separater graphs for onprem and cloud
    try: 
        axs[i].set_title("CLOUD " + "wait " + str(jobs_wait['wait_time'][i]) +  " arr " + str(jobs_wait['arrival_rate'][i]), fontsize=10)
        axs[i].hist(cloud_waits[i])
        print("Mask")
        print(jobs_wait['arrival_mask'][i])
    except Exception as e: 
        continue 
plt.show()

#time.sleep(100000)
'''
# TODO: Find avergage of migrated values vs non migrated values

waits = {}
for i in range(len(jobs_wait)):
    try: 
        onprem_jobs = [jobs_wait['wait_times'][i][j] * jobs_wait['arrival_mask'][i][j] for j in range(len(jobs_wait['wait_times'][i]))]
        local_wait = sum(onprem_jobs)/sum(jobs_wait['arrival_mask'][i])

        cloud_jobs = [jobs_wait['wait_times'][i][j] * (-1 * (jobs_wait['arrival_mask'][i][j] - 1)) for j in range(len(jobs_wait['wait_times'][i]))]
        cloud_wait = sum(cloud_jobs)/(len(cloud_jobs)-sum(jobs_wait['arrival_mask'][i]))

        waits["wait " + str(jobs_wait['wait_time'][i]) +  " arr " + str(jobs_wait['arrival_rate'][i])] = (local_wait, cloud_wait)

        #print(cloud_jobs)
        #print("wait_times")
        print(jobs_wait['wait_times'][i])
        #runtime_diff
        #print("migrated_times")
        #print(migrated_average)
    except Exception as e:
        continue 

#jobs_df_ar_2 = jobs_df.filter()
#for i in range(3):
    

print("(local_job_avg_wait, cloud_job_avg_wait)")
for wait in waits: 
    print((wait, waits[wait]))
#print(waits)

fig, axs = plt.subplots(nrows=1, ncols=len(jobs_wait), figsize=(50, 3))  #figsize=(15, 3))

for i in range(len(jobs_df)):
    #TODO: Incorporate tracing for efficiently
    #import pdb; pdb.set_trace()
    
    # TODO: Plot separater graphs for onprem and cloud
    try: 
        axs[i].set_title("wait " + str(jobs_wait['wait_time'][i]) +  " arr " + str(jobs_wait['arrival_rate'][i]), fontsize=10)
        axs[i].hist(jobs_wait['wait_times'][i])
        print("Mask")
        print(jobs_wait['arrival_mask'][i])
    except Exception as e: 
        continue 
#print(jobs_df['arrival_mask'][0])
#print(jobs_df['arrival_mask'][1])
#plt.show()

fig, axs = plt.subplots(nrows=1, ncols=len(jobs_wait), figsize=(50, 3))  #figsize=(15, 3))

for i in range(len(jobs_df)):
    #TODO: Incorporate tracing for efficiently
    #import pdb; pdb.set_trace()
    
    # TODO: Plot separater graphs for onprem and cloud
    try: 
        axs[i].set_title("wait " + str(jobs_wait['wait_time'][i]) +  " arr " + str(jobs_wait['arrival_rate'][i]), fontsize=10)
        axs[i].hist(jobs_wait['wait_times'][i])
        print("Mask")
        print(jobs_wait['arrival_mask'][i])
    except Exception as e: 
        continue 
plt.show()

#jobs_df = all_jobs_original
'''

In [ ]:
#print(plot_jobs.GPUS_PER_NODE)
#time.sleep(1000)
gpus_per_node = 32
avoid_congestion = True #False 
print(gpus_per_node)
data = job_logs.view_real_arrival_times(event_number=logs, scale=0.5, plot_sweep=True, get_data=False, dim=(-100, 100, 0, 100), ratio=(5, 75), gpus_per_node = gpus_per_node, avoid_congestion=avoid_congestion)#True)#False)

if data and len(data) == 3:
    jobs, events, submissions = data
    
    #print("JOBS")
    #print(jobs)
    #print("EVENTS")
    #print(events)
    #print("SUBMISSIONS")
    #print(submissions)
# TODO: Specify each subplot size
# TODO: Specify the yaxis min and yaxis max

#job_logs.analyze_sweep(logs, graph=True)

In [ ]:
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df = metrics_df.transpose()
#metrics_df

#metrics_df
metrics_df.keys()
#metrics_df.values()
t_metrics_df = metrics_df.transpose()
t_metrics_df= t_metrics_df.rename(columns={'cpus_per_node': 'gpus_per_node'})
#t_metrics_df= t_metrics_df.rename(columns={'cpus_per_node': 'num_gpus'})
t_metrics_df.keys()

In [ ]:
def new_plotting_function():
    return 0 

def skyburst_plotting_function(): 
    return 0 

def set_plotting_setting(ax):
    plt.rcParams.update({'font.size': 15})
    ax.grid(True, which='both')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    #ax.tick_params(bottom=False, left=False)
    ax.tick_params(bottom=True, left=False)
    ax.set_axisbelow(True)

label_dict = {
    'avg_jct': 'Avg. JCT (hr)',
    'cost_mult': '% Cost Savings\nover No Wait',
    'cost_diff': 'Cost Savings\nover No Wait',
    'cluster_size': 'Cluster Size (# Nodes)',
    'norm_system_utilization': 'System Utilization',
    'system_utilization': 'System Utilization',
    'cluster_utilization': 'Cluster Utilization',
    'total_cloud_cost': 'Cloud Cost',
    'arrival_rate': 'Arrival Rate',
}

legend_dict = {
    'constant': 'Constant',
    'linear_runtime': 'Runtime',
    'linear_cost': 'Cost',
    'zero': 'No Wait',
    'linear_runtime_filter_cpu': 'Runtime-Preempt-CPU'
}

def join_baseline(df, 
                          x_axis: str,
                          y_axis: list = ['cost_mult', 'avg_jct'],
                          df_filter: dict = {},
                          baseline_filter: dict={'waiting_policy': 'zero',},
                          groupby_values=['waiting_policy', 'waiting_factor'],
                          normalize_x_axis=False):
    if isinstance(y_axis, str):
        y_axis = [y_axis]
    fig, axs = plt.subplots(nrows=1, ncols=len(y_axis), figsize=(5*len(y_axis), 3.5))
    
    for k,v in df_filter.items():
        df = df[df[k]==v]
       
    baseline_df = df
    for k,v in baseline_filter.items():
        assert not isinstance(v, list)
        baseline_df = baseline_df[baseline_df[k]==v]
        #df = df[df[k]!=v]
    diff_df = pd.merge(df, baseline_df, left_on=x_axis,right_on=x_axis)
    return diff_df
    
def get_default(df, 
                          x_axis: str,
                          y_axis: list = ['cost_mult', 'avg_jct'],
                          df_filter: dict = {},
                          baseline_filter: dict={'waiting_policy': 'zero',},
                          groupby_values=['waiting_policy', 'waiting_factor'],
                          normalize_x_axis=False):
    if isinstance(y_axis, str):
        y_axis = [y_axis]
        
    fig, axs = plt.subplots(nrows=1, ncols=len(y_axis), figsize=(5*len(y_axis), 3.5))
    
    for k,v in df_filter.items():
        print(k)
        print(v)
        print(df[k])
        #print(df[k]==v)
        #df[k] = df[k].apply(lambda x: [0, 0, 0] if len(x) != 3 else x)
        #print(df[k].eq(v))
        
        
        mask = df[k].apply(lambda x: x == v)
        print(mask)
        if isinstance(v, list):
            df = df[mask]
            #df[k].eq(v)
            #df[k] = df[k].apply(lambda x: [0, 0, 0] if len(x) != 3 else x)
            #df = df[df[k].eq(v)]
            
        #df = df[df[k]==v]
        
    return df
    
def get_baseline(df, 
                          x_axis: str,
                          y_axis: list = ['cost_mult', 'avg_jct'],
                          df_filter: dict = {},
                          baseline_filter: dict={'waiting_policy': 'zero',},
                          groupby_values=['waiting_policy', 'waiting_factor'],
                          normalize_x_axis=False):
    if isinstance(y_axis, str):
        y_axis = [y_axis]
    fig, axs = plt.subplots(nrows=1, ncols=len(y_axis), figsize=(5*len(y_axis), 3.5))
    
    for k,v in df_filter.items():        
        mask = df[k].apply(lambda x: x == v)
        if isinstance(v, list):
            df = df[mask]
    
    baseline_df = df
    
    for k,v in baseline_filter.items():
        #assert not isinstance(v, list)
        #baseline_df = baseline_df[baseline_df[k]==v]
        mask = baseline_df[k].apply(lambda x: x == v)
        baseline_df = baseline_df[mask]
        baseline_df = df[mask]
        #print(mask)
        #if isinstance(v, list):
            #df[k].eq(v)
            #df[k] = df[k].apply(lambda x: [0, 0, 0] if len(x) != 3 else x)
            #df = df[df[k].eq(v)]
            
        #df = df[df[k]!=v]
    
    diff_df = pd.merge(df, baseline_df, left_on=x_axis,right_on=x_axis)
    
    if normalize_x_axis:
        if x_axis == 'cluster_size':
            # Hardcoded in Philly trace, precomputed ahead of time
            if df['dataset'].iloc[0] == 'philly':
                total_job_volume = 1155998.77277777
                job_makespan = 2559.3205555555555
            elif df['dataset'].iloc[0] == 'helios':
                total_job_volume = 1853756.1241666232
                job_makespan = 4651.911388888889
            diff_df['norm_system_utilization'] = total_job_volume/(job_makespan*diff_df['cluster_size']*df['gpus_per_node'].iloc[0])
            x_axis = 'norm_system_utilization'
        elif x_axis == 'arrival_rate':
            avg_job_volume_rate = diff_df['arrival_rate'] * np.mean(df['num_gpus'].iloc[0]* df['runtime'].iloc[0])
            #print(avg_job_volume_rate)
            #print(df['cluster_size'].iloc[0])
            #print(df['gpus_per_node'].iloc[0])
            #print(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            diff_df['norm_system_utilization'] = avg_job_volume_rate/(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            x_axis = 'norm_system_utilization'
        
    #return baseline_df
    return diff_df

def simulator_plotting_fn(df, 
                          x_axis: str,
                          y_axis: list = ['cost_mult', 'avg_jct'],
                          df_filter: dict = {},
                          baseline_filter: dict={'waiting_policy': 'zero',},
                          groupby_values=['waiting_policy', 'waiting_factor'],
                          normalize_x_axis=False,
                          fig_ratio=(5, 3.5)):
    if isinstance(y_axis, str):
        y_axis = [y_axis]
    fig, axs = plt.subplots(nrows=1, ncols=len(y_axis), figsize=(fig_ratio[0]*len(y_axis), fig_ratio[1]))
    
    if len(y_axis) == 1: 
        if not isinstance(axs, list):
            axs = [axs]
    print(axs)
    #time.sleep(1000)
    
    '''
    for k,v in df_filter.items():
        if isinstance(v, list):
            df = df[df[k]==v]
    '''

    for k,v in df_filter.items():        
        mask = df[k].apply(lambda x: x == v)
        if isinstance(v, list):
            df = df[mask]
       
    baseline_df = df
    
    for k,v in baseline_filter.items():
        assert not isinstance(v, list)
        baseline_df = baseline_df[baseline_df[k]==v]
        #df = df[df[k]!=v]
        
    diff_df = pd.merge(df, baseline_df, left_on=x_axis,right_on=x_axis)
    
    if normalize_x_axis:
        if x_axis == 'cluster_size':
            # Hardcoded in Philly trace, precomputed ahead of time
            if df['dataset'].iloc[0] == 'philly':
                total_job_volume = 1155998.77277777
                job_makespan = 2559.3205555555555
            elif df['dataset'].iloc[0] == 'helios':
                total_job_volume = 1853756.1241666232
                job_makespan = 4651.911388888889
            diff_df['norm_system_utilization'] = total_job_volume/(job_makespan*diff_df['cluster_size']*df['gpus_per_node'].iloc[0])
            x_axis = 'norm_system_utilization'
        elif x_axis == 'arrival_rate':
            # Volume rate => runtime must be in either hours or seconds 
            '''
            Verification: 
            - Arrival_Rate = VERIFIED ~ error 
            - Num_gpus = Verified
            - Runtime = Verified 
            '''
            avg_job_volume_rate = diff_df['arrival_rate'] * np.mean(df['num_gpus'].iloc[0]* df['runtime'].iloc[0])
            df['z_mean_runtime'] = np.mean(df['num_gpus'].iloc[0]* df['runtime'].iloc[0])
            df['z_cluster_nodes'] = (df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            #print(avg_job_volume_rate)
            #print(df['cluster_size'].iloc[0])
            #print(df['gpus_per_node'].iloc[0])
            #print(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            diff_df['norm_system_utilization'] = avg_job_volume_rate/(df['cluster_size'].iloc[0]*df['gpus_per_node'].iloc[0])
            x_axis = 'norm_system_utilization'
    
    print(df)
    
    def cost_multiplier(row):
        baseline_cost = row['total_cloud_cost_y']
        cost = row['total_cloud_cost_x']
        if baseline_cost == 0 and cost==0:
            return 0
        elif baseline_cost <=10000:
            # Small cloud cost for No wait
            # Savings over small cloud cost is negligible for organizations.
            return 0
        elif baseline_cost == 0 and cost>0:
            return 100
        return 100* (1 - (cost/baseline_cost))
    
    def cost_difference(row):
        baseline_cost = row['total_cloud_cost_y']
        cost = row['total_cloud_cost_x']
        return baseline_cost - cost
    
    diff_df['cost_mult'] = diff_df.apply(cost_multiplier, axis=1)
    diff_df['cost_diff'] = diff_df.apply(cost_difference, axis=1)
    groupby_values = [f'{g}_x' for g in groupby_values]
    mod_y_axis = [f'{y}_x' if y!='cost_mult' and y!='cost_diff' else y for y in y_axis]
    import itertools
    markers = itertools.cycle(('v', '^','.', 'o', '*',',', '+',)) 
    for idx, (label, grp) in enumerate(diff_df.groupby(groupby_values)):
        marker = next(markers)
#         if 'waiting_policy' in groupby_values[0]:
#             label = [legend_dict[label[0]]] + list(label[1:])
#             print(label)
        for ax_idx, ax in enumerate(axs):           
            grp.plot(x = x_axis, y = mod_y_axis[ax_idx],ylabel=label_dict[y_axis[ax_idx]], \
                     xlabel=label_dict[x_axis], marker=marker, ax = ax, label = label, legend=None)
    
    
    for ax in axs:
        set_plotting_setting(ax)
    #axs[1].set_xlim(right=36, left=20)
    lines, labels = ax.get_legend_handles_labels()
    fig.legend(lines, labels, ncol=len(labels), \
               bbox_to_anchor=(0, 0.92, 1, 0.2),loc='upper center')
    plt.tight_layout()
    plt.show()  

In [ ]:
jobs_df

In [ ]:
default_df = get_default(jobs_df, x_axis='arrival_rate', y_axis=['avg_jct'], \
                           #df_filter={'cpu_dist': [0.2, 0.4, 0.4]}, \
                         #df_filter={'cpu_dist': [0.0, 0.5, 0.5]}, \
                         df_filter={'cpu_dist': [0.6, 0.25, 0.15]}, \
                          #df_filter={'wait_time': 1.0}, \
                      #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 1.0}, \
                          #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0.0}, \
                         baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0}, \
                           groupby_values=['waiting_policy'], normalize_x_axis=True)


baseline_df = get_baseline(jobs_df, x_axis='arrival_rate', y_axis=['avg_jct'], \
                           #df_filter={'cpu_dist': [0.2, 0.4, 0.4]}, \
                           #df_filter={'cpu_dist': [0.0, 0.5, 0.5]}, \
                           df_filter={'cpu_dist': [0.6, 0.25, 0.15]},
                      #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 1.0}, \
                           baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0}, \
                           groupby_values=['waiting_policy'], normalize_x_axis=True)


merged_df = join_baseline(jobs_df, x_axis='arrival_rate', y_axis=['avg_jct', 'total_cloud_cost'], \
                          #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 1.0}, \
                          baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0}, \
                      groupby_values=['waiting_policy'], normalize_x_axis=True)
default_df, baseline_df, merged_df

In [ ]:
#jobs_df

In [ ]:
#dists = [[0.2, 0.4, 0.4], [0, 0.5, 0.5], [0, 0, 1]]
dists = [[0.6, 0.25, 0.15]]#[[0, 0.5, 0.5]]
scale = 4
for dist in dists:#[[0.2, 0.4, 0.4], [0, 0.5, 0.5], [0, 0, 1]]:
    #print(f'dist: {dist}')
    simulator_plotting_fn(jobs_df, x_axis='arrival_rate', \
                          #y_axis=['total_cloud_cost'], 
                          y_axis=['avg_jct', 'cluster_utilization'],\
                          #y_axis=['avg_jct'],\
                          #'avg_jct', 'total_cloud_cost'], \
                          df_filter={'cpu_dist': dist}, 
                          #baseline_filter= {'wait_time': 1.0}, \
                          #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0.0}, \
                          baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 0.0}, \
                          #baseline_filter= {'waiting_policy': 'fifo_wait', 'wait_time': 1.0}, \
                          #groupby_values=['waiting_policy', 'wait_time'], normalize_x_axis=True, \
                          groupby_values=['wait_time'], normalize_x_axis=True, \
                          fig_ratio=(5*scale, 3.5*scale))
    
# TODO: Add spec to increase plot size
#simulator_plotting_fn(t_metrics_df, x_axis='arrival_rate', y_axis=['avg_jct'], \
#                      groupby_values=['waiting_policy'], normalize_x_axis=True)

In [ ]:
hyp = copy.deepcopy(sampled_jobs.DEFAULT_HYPERPARAMETERS)

hyp['arrival_rate'] = 10
hyp['batch_time'] = 600
hyp['total_jobs'] = 5
hyp['mean_duration'] = 60
arrived_jobs, arrivals = sampled_jobs.generate_jobs(hyp)

#print(arrived_jobs)
#print(arrivals)

In [ ]:
## HYPERPARAMETER
'''
Vary arrival rate
    - CPU jobs for Philly/Helios
        
- Mimic GPU jobs ~ with CPU
    - vary # jobs per minute 
    - test contant waiting for baseline
    - 0(Baseline), 0.25, 0.5, 1 ratio of runtime for waiting policy 
        - 
    
- Plot Cluster Utilization 
- Plot Cloud Cost 
'''